In [1]:
import cv2
import mediapipe as mp
import os
import time
import pandas as pd

In [2]:
# Khởi tạo Mediapipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

### Thử nghiệm với Mediapipe Face

In [3]:
# Thư mục chứa ảnh
image_folder = "../tool/frames_extraction/20250307_222846"

temp_folder = "mediapipe_face_detection_temp"

# Tạo thư mục "temp" nếu chưa tồn tại
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

# Danh sách lưu kết quả
results_list = []

detected_image_count = 0
total_frame_count = 0

# Duyệt qua tất cả ảnh trong folder
for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Chỉ lấy file ảnh
        image_path = os.path.join(image_folder, filename)

        # Đọc ảnh
        image = cv2.imread(image_path)
        if image is None:
            continue

        total_frame_count += 1

        # Chuyển ảnh sang RGB (vì Mediapipe dùng RGB thay vì BGR)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Bắt đầu đo thời gian
        start_time = time.time()
        
        # Phát hiện khuôn mặt
        results = face_detection.process(image_rgb)
        
        # Kết thúc đo thời gian
        end_time = time.time()
        processing_time = end_time - start_time  # Thời gian xử lý (giây)
        
        # Số khuôn mặt phát hiện được
        num_faces = len(results.detections) if results.detections else 0

        # Vẽ bounding box lên ảnh
        if results.detections:
            for idx, detection in enumerate(results.detections):
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                                int(bboxC.width * iw), int(bboxC.height * ih)
                
                # Cắt ảnh chỉ giữ lại khuôn mặt
                face_crop = image[max(0, y):min(ih, y+h), max(0, x):min(iw, x+w)]

                # Đường dẫn lưu ảnh đã cắt
                output_path = os.path.join(temp_folder, f"{filename}_face{idx}.jpg")
                cv2.imwrite(output_path, face_crop)

                detected_image_count += 1
        
        # Lưu kết quả
        results_list.append({
            "image": filename,
            "num_faces": num_faces,
            "processing_time": processing_time
        })

In [4]:
# Chuyển kết quả thành DataFrame
df = pd.DataFrame(results_list)

In [5]:
print("Số frame có thể phát hiện khuôn mặt:", detected_image_count)
print("Tổng số frame:", total_frame_count)
print("Tỉ lệ phát hiện khuôn mặt:", detected_image_count / total_frame_count * 100, "%")

average_processing_time = df['processing_time'].mean()
max_processing_time = df['processing_time'].max()
min_processing_time = df['processing_time'].min()
print(f"⏱️ Thời gian xử lý nhanh nhất: {min_processing_time:.4f} giây")
print(f"⏱️ Thời gian xử lý lâu nhất: {max_processing_time:.4f} giây")
print(f"⏱️ Thời gian xử lý trung bình cho mỗi ảnh: {average_processing_time:.4f} giây")

Số frame có thể phát hiện khuôn mặt: 344
Tổng số frame: 356
Tỉ lệ phát hiện khuôn mặt: 96.62921348314607 %
⏱️ Thời gian xử lý nhanh nhất: 0.0000 giây
⏱️ Thời gian xử lý lâu nhất: 0.0206 giây
⏱️ Thời gian xử lý trung bình cho mỗi ảnh: 0.0064 giây


### Thử nghiệm với Haar Cascade

In [6]:
# Load mô hình Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Thư mục chứa ảnh đầu vào
image_folder = "../tool/frames_extraction/20250307_222846"

# Thư mục chứa ảnh đầu ra (chỉ chứa khuôn mặt)
temp_folder = "haar_cascade_face_detection_temp"

# Tạo thư mục "temp" nếu chưa tồn tại
if not os.path.exists(temp_folder):
    os.makedirs(temp_folder)

# Danh sách lưu kết quả
results_list = []

detected_image_count = 0
total_frame_count = 0

# Duyệt qua tất cả ảnh trong folder
for filename in os.listdir(image_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg')):  # Chỉ lấy file ảnh
        image_path = os.path.join(image_folder, filename)

        # Đọc ảnh
        image = cv2.imread(image_path)
        if image is None:
            continue

        total_frame_count += 1

        # Chuyển ảnh sang grayscale (Haar Cascade yêu cầu ảnh grayscale)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Bắt đầu đo thời gian
        start_time = time.time()
        
        # Phát hiện khuôn mặt
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        
        # Kết thúc đo thời gian
        end_time = time.time()
        processing_time = end_time - start_time  # Thời gian xử lý (giây)
        
        # Số khuôn mặt phát hiện được
        num_faces = len(faces)

        # Lưu các khuôn mặt đã cắt
        for idx, (x, y, w, h) in enumerate(faces):
            # Cắt ảnh chỉ giữ lại khuôn mặt
            face_crop = image[max(0, y):min(image.shape[0], y+h), max(0, x):min(image.shape[1], x+w)]

            # Đường dẫn lưu ảnh đã cắt
            output_path = os.path.join(temp_folder, f"{filename}_face{idx}.jpg")
            cv2.imwrite(output_path, face_crop)

            detected_image_count += 1

        # Lưu kết quả
        results_list.append({
            "image": filename,
            "num_faces": num_faces,
            "processing_time": processing_time
        })

In [7]:
# Chuyển kết quả thành DataFrame
df = pd.DataFrame(results_list)

print("Số frame có thể phát hiện khuôn mặt:", detected_image_count)
print("Tổng số frame:", total_frame_count)
print("Tỉ lệ phát hiện khuôn mặt:", detected_image_count / total_frame_count * 100, "%")

average_processing_time = df['processing_time'].mean()
max_processing_time = df['processing_time'].max()
min_processing_time = df['processing_time'].min()

print(f"⏱️ Thời gian xử lý nhanh nhất: {min_processing_time:.4f} giây")
print(f"⏱️ Thời gian xử lý lâu nhất: {max_processing_time:.4f} giây")
print(f"⏱️ Thời gian xử lý trung bình cho mỗi ảnh: {average_processing_time:.4f} giây")

Số frame có thể phát hiện khuôn mặt: 172
Tổng số frame: 356
Tỉ lệ phát hiện khuôn mặt: 48.31460674157304 %
⏱️ Thời gian xử lý nhanh nhất: 0.0312 giây
⏱️ Thời gian xử lý lâu nhất: 0.2627 giây
⏱️ Thời gian xử lý trung bình cho mỗi ảnh: 0.0480 giây
